In [ ]:
# Realizar as importações
import numpy as np
import matplotlib.pyplot as plt
import cv2

input_path = '../images/'
output_path = 'output/'

In [ ]:
# Funções úteis

def mostrar(image : np.ndarray, cmap1 = None, title = "Imagem", orig : np.ndarray = None, cmap2 = None):

    if orig is not None:
        plt.figure(figsize=(20,10))
        plt.subplot(1,2,2)
        plt.imshow(orig, cmap=cmap2)
        plt.title(title + " Original")
        plt.axis('off')
        plt.subplot(1,2,1)
        plt.imshow(image, cmap=cmap1)
        plt.axis('off')
        plt.show()
    else:
        plt.figure(figsize=(10,10))
        plt.imshow(image, cmap=cmap1)
        plt.axis('off')
        plt.show()

def mostrar_histograma(image : np.ndarray, title = "Histograma", orig : np.ndarray = None):
    if orig is not None:
        plt.figure(figsize=(20,10))
        plt.subplot(1,2,2)
        plt.hist(orig.ravel(), bins=256, range=(0, 256))
        plt.xlim([-1, np.max(orig) + 10])
        plt.title(title + " Original")
        plt.subplot(1,2,1)
        plt.hist(image.ravel(), bins=256, range=(0, 256))
        plt.xlim([-1, np.max(image) + 10])
        plt.title(title)
        plt.show()
    else:
        plt.hist(image.ravel(), bins=256, range=(0, 256))
        plt.title(title)
        plt.xlim([-1, np.max(image) + 10])
        plt.show()


# 1.1 - Esboço a Lápis

Implementar um efeito de esboço a lápis em uma imagem por meio dos seguintes passos:

(i) converter a imagem colorida para níveis de cinza,

(ii) aplicar um filtro de desfoque gaussiano,(por exemplo, com uma máscara de 21×21 pixels) para suavizar os detalhes da imagem,

(iii) dividir a imagem em tons de cinza pela versão desfocada para realçar os contornos

In [ ]:
nome = 'watch'
imagem = cv2.imread(input_path + nome + '.png', cv2.IMREAD_COLOR)
imagem_orig = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
imagem_orig = imagem_orig.astype(np.int16)

imagem_gray = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
imagem_gray = imagem_gray.astype(np.int16)

t_mask = 21
imagem_desfocada= cv2.GaussianBlur(imagem_gray, (t_mask, t_mask), 0, borderType=cv2.BORDER_REFLECT)
imagem_desfocada = imagem_desfocada.astype(np.int16)

plt.figure(figsize=(30, 10))
plt.subplot(1, 3, 1)
plt.imshow(imagem_orig)
plt.title('Imagem Original')
plt.axis('off')
plt.subplot(1, 3, 2)
plt.imshow(imagem_gray, cmap='gray', vmin= 0, vmax= 255)
plt.title('Imagem em Tons de Cinza')
plt.axis('off')
plt.subplot(1, 3, 3)
plt.imshow(imagem_desfocada, cmap='gray', vmin= 0, vmax= 255)
plt.title('Imagem Desfocada')
plt.axis('off')
plt.show()

In [ ]:
imagem_gray = imagem_gray + 1
imagem_desfocada = imagem_desfocada + 1
imagem_dividida = imagem_gray / imagem_desfocada
cv2.normalize(imagem_dividida, imagem_dividida, 0, 255, cv2.NORM_MINMAX)
imagem_dividida = imagem_dividida.astype(np.uint8)

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(imagem_dividida, cmap='gray', vmin= 0, vmax= 255)
plt.title('Imagem Dividida')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.hist(imagem_dividida.ravel(), bins=256, range=(0, 256))
plt.xlim([-1, np.max(imagem_dividida) + 10])
plt.title('Histograma da Imagem Dividida')
plt.show()

imagem_clipada = np.clip(imagem_dividida, 10, 40) #watch 10, 40
plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(imagem_clipada, cmap='gray', vmin= 0, vmax= 255)
plt.title('Imagem Clipada')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.hist(imagem_clipada.ravel(), bins=256, range=(0, 256))
plt.xlim([-1, np.max(imagem_clipada) + 10])
plt.title('Histograma da Imagem Clipada')
plt.show()

imagem_final = 0
imagem_final = cv2.normalize(imagem_clipada, imagem_final, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(imagem_final, cmap='gray', vmin= 0, vmax= 255)
plt.title('Imagem Final')
plt.axis('off')
cv2.imwrite(output_path + "1_" + "Lapis" + ".png", imagem_final, [cv2.IMWRITE_PNG_COMPRESSION, 0])
plt.subplot(1, 2, 2)
plt.hist(imagem_final.ravel(), bins=256, range=(0, 256))
plt.xlim([-1, np.max(imagem_final) + 10])
plt.title('Histograma da Imagem Final')
plt.show()

# 1.2 Ajuste de Brilho
Aplicar a correção gama para ajustar o brilho de uma imagem monocromatica A de entrada e gerar
uma imagem monocromatica B de saída. A transformação pode ser realizada

(i) convertendo-se as intensidades dos pixels para o intervalo de [0,255] para [0,1]

(ii) aplicando-se a equação B = A^(1/y) e 

(iii) convertendo-se os valores resultantes de volta para o intervalo [0,255]. 

Realizar a correção com diferentes valores de y.

(a) imagem (b) y = 1.5 (c) y = 2.5 (d) y = 3.5

In [ ]:
nome = 'house'
imagem_gray = cv2.imread(input_path + nome + '.png', cv2.IMREAD_GRAYSCALE)
imagem_gray = imagem_gray.astype(np.float32)

imagens = []
gamas = [0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0, 5.0, 9.0]
for gama in gamas:
    imagem_gama = cv2.normalize(imagem_gray, None, 0, 1, cv2.NORM_MINMAX)
    imagem_gama = cv2.pow(imagem_gray, 1/gama)
    imagem_gama = cv2.normalize(imagem_gama, None, 0, 255, cv2.NORM_MINMAX)
    imagem_gama = imagem_gama.astype(np.uint8)
    imagens.append(imagem_gama)

for i in range(3):
    plt.figure(figsize=(30, 10))
    for j in range(3):
        plt.subplot(1, 3, j + 1)
        plt.imshow(imagens[3*i + j], cmap='gray', vmin=0, vmax=255)
        plt.title('Imagem Original' if 3*i + j == 3 else f'Gamma = {gamas[3*i + j]}')
        plt.axis('off')
        cv2.imwrite(output_path + "2_" + ('Imagem Original' if 3*i + j == 3 else f'Gamma {gamas[3*i + j]}').replace(" ", "_") + ".png", imagens[3*i + j], [cv2.IMWRITE_PNG_COMPRESSION, 0])
    plt.show()



# 1.3 Mosaico
Construir um mosaico de 4×4 blocos a partir de uma imagem monocromática. A disposição dos
blocos deve seguir a numeração mostrada na figura (c).

In [ ]:
nome = 'butterfly'
imagem = cv2.imread(input_path + nome + '.png', cv2.IMREAD_GRAYSCALE)
shape = imagem.shape
print(shape)
imagem_gray = imagem.astype(np.int16)

In [ ]:
#Disposição dos blocos:
destino = [6, 10, 3, 12, 15, 0, 11, 4, 7, 14, 1, 8, 2, 9, 13, 5]
destino = np.array(destino)
origem = np.arange(16)

cortes_destino = [((shape[0]//4) * (i//4), (shape[0]//4) * ((i//4) + 1) , (shape[1]//4) * (i%4), (shape[1]//4) * ((i%4) + 1) ) for i in destino]
cortes_destino = np.array(cortes_destino)
cortes_destino = cortes_destino.astype(np.int16)

cortes_origem = [((shape[0]//4) * (i//4), (shape[0]//4) * ((i//4) + 1) , (shape[1]//4) * (i%4), (shape[1]//4) * ((i%4) + 1) ) for i in origem]
cortes_origem = np.array(cortes_origem)
cortes_origem = cortes_origem.astype(np.int16)

imagem_final = np.zeros(imagem_gray.shape, dtype=np.int16)
for i in range(len(destino)):
    imagem_final[cortes_destino[i][0]:cortes_destino[i][1], cortes_destino[i][2]:cortes_destino[i][3]] = imagem_gray[cortes_origem[i][0]:cortes_origem[i][1], cortes_origem[i][2]:cortes_origem[i][3]]

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(imagem_final, cmap='gray', vmin=0, vmax=255)
plt.title('Imagem Remontada')
plt.axis('off')
cv2.imwrite(output_path + "3_" + "Mosaico" + ".png", imagem_final, [cv2.IMWRITE_PNG_COMPRESSION, 0])
plt.subplot(1, 2, 2)
plt.imshow(imagem_gray, cmap='gray', vmin= 0, vmax= 255)
plt.title('Imagem Original')
plt.axis('off')

# 1.4 Alteração de Cores
Implementar um filtro para simular o efeito de fotografias antigas por meio da aplicação de uma
transformação linear nas cores da imagem. Cada pixel deve ser multiplicado pela seguinte matriz
de transformação:

In [ ]:
mat =  [[0.393, 0.769, 0.189],
        [0.349, 0.686, 0.168],
        [0.272, 0.534, 0.131]]
mat = np.array(mat)
mat = mat.astype(np.float32)

In [ ]:
nome = 'watch'
imagem = cv2.imread(input_path + nome + '.png', cv2.IMREAD_COLOR)
imagem_orig = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
imagem_orig = imagem_orig.astype(np.int16)

imagem_final = cv2.transform(imagem_orig, mat)
cv2.normalize(imagem_final, imagem_final, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.title('Imagem Antiga')
plt.imshow(imagem_final, vmin= 0, vmax= 255)
plt.axis('off')
cv2.imwrite(output_path + "4_" + "Antiga" + ".png", imagem_final, [cv2.IMWRITE_PNG_COMPRESSION, 0])
plt.subplot(1, 2, 2)
plt.imshow(imagem_orig, vmin= 0, vmax= 255)
plt.title('Imagem Original')
plt.axis('off')

# 1.5 Transformação de Imagens Coloridas

## (a) 
Dada uma imagem colorida no formato RGB, alterar a imagem conforme as seguintes operações:

$R’ = 0.393R + 0.769G + 0.189B$

$G’ = 0.349R + 0.686G + 0.168B$

$B’ = 0.272R + 0.534G + 0.131B$

Após a transformação, caso $R’$, $G’$ ou $B’$ tenha valor maior do que 255, ele deve ser limitado em 255.

In [ ]:
nome = 'watch'
imagem = cv2.imread(input_path + nome + '.png', cv2.IMREAD_COLOR)
imagem_orig = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
imagem_orig = imagem_orig.astype(np.int16)

def transf(imagem : np.ndarray) -> np.ndarray:
    nova = np.zeros(imagem.shape, dtype=np.int16)
    nova[:, :, 0] = imagem[:, :, 0] * 0.393 + imagem[:, :, 1] * 0.769 + imagem[:, :, 2] * 0.189
    nova[:, :, 1] = imagem[:, :, 0] * 0.349 + imagem[:, :, 1] * 0.686 + imagem[:, :, 2] * 0.168
    nova[:, :, 2] = imagem[:, :, 0] * 0.272 + imagem[:, :, 1] * 0.534 + imagem[:, :, 2] * 0.131
    cv2.normalize(nova, nova, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    nova = nova.astype(np.uint8)
    return nova

imagem_final = transf(imagem_orig)

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.title('Imagem Transformada')
plt.imshow(imagem_final, vmin= 0, vmax= 255)
plt.axis('off')
cv2.imwrite(output_path + "5_" + "Transformada" + ".png", imagem_final, [cv2.IMWRITE_PNG_COMPRESSION, 0])

plt.subplot(1, 2, 2)
plt.imshow(imagem_orig, vmin= 0, vmax= 255)
plt.title('Imagem Original')
plt.axis('off')

## (b) 
Dada uma imagem colorida no formato RGB, alterar a imagem tal que ela contenha apenas
uma banda de cor, cujos valores s ̃ao calculados pela m ́edia ponderada:

$I = 0.2989R + 0.5870G + 0.1140B$

In [ ]:
nome = 'watch'
imagem = cv2.imread(input_path + nome + '.png', cv2.IMREAD_COLOR)
imagem_orig = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
imagem_orig = imagem_orig.astype(np.int16)

shape = imagem_orig.shape
shape = (shape[0], shape[1], 1)
imagem_final = np.zeros(shape, dtype=np.int16)
imagem_final = 0.299 * imagem_orig[:, :, 0] + 0.587 * imagem_orig[:, :, 1] + 0.114 * imagem_orig[:, :, 2]
cv2.normalize(imagem_final, imagem_final, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
imagem_final = imagem_final.astype(np.uint8)

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.title('Imagem Monocromática')
plt.imshow(imagem_final, cmap='gray', vmin= 0, vmax= 255)
plt.axis('off')
cv2.imwrite(output_path + "5_" + "Monocromatica" + ".png", imagem_final, [cv2.IMWRITE_PNG_COMPRESSION, 0])

plt.subplot(1, 2, 2)
plt.imshow(imagem_orig, vmin= 0, vmax= 255)
plt.title('Imagem Original')
plt.axis('off')


# 1.6 Planos de Bits

In [ ]:
mascara = [2**i for i in range(0, 8)]
mascara = np.array(mascara) 

nome = "peppers"
imagem_orig = cv2.imread(input_path + nome + ".png", cv2.IMREAD_GRAYSCALE)

plans = []
for i in range(len(mascara)):
    plano = imagem_orig & mascara[i]
    plano = plano >> i
    plano = plano.astype(np.uint8)
    plano = plano * 255
    plans.append(plano)
    
plans.append(imagem_orig)

for i in range(3):
    plt.figure(figsize=(30, 10))
    for j in range(3):
        plt.subplot(1, 3, j + 1)
        plt.imshow(plans[3*i + j], cmap="gray")
        plt.title("Imagem Original" if 3*i + j == 8 else f"Plano de bit {3*i + j}")
        plt.axis("off")
        cv2.imwrite(output_path + "6_" + ("Imagem Original" if 3*i + j == 8 else f"Plano de bit {3*i + j}").replace(" ", "_") + ".png", imagens[3*i + j], [cv2.IMWRITE_PNG_COMPRESSION, 0])
    plt.show()

# 1.7 Combinação de Imagens

In [ ]:
nome1 = "peppers"
imagem1 = cv2.imread(input_path + nome1 + ".png", cv2.IMREAD_GRAYSCALE)
shape1 = imagem1.shape
print(shape1)

nome2 = "house"
imagem2 = cv2.imread(input_path + nome2 + ".png", cv2.IMREAD_GRAYSCALE)
shape2 = imagem2.shape
print(shape2)

mostrar(imagem2, title= "city", orig=imagem1, cmap1="gray", cmap2="gray")
imagens = [0] * 9

for i in range(1,10):
    imagens[i-1] = imagem1*(i/10) + imagem2*((10-i)/10)

for i in range(3):
    plt.figure(figsize=(30, 10))
    for j in range(3):
        plt.subplot(1, 3, j + 1)
        plt.imshow(imagens[3*i + j], cmap="gray")
        plt.title(f"{(3*i + j + 1)/10:.1f} * A + {(10 - (3*i + j + 1))/10:.1f} * B")
        plt.axis("off")
        cv2.imwrite(output_path + "7_" + (f"{(3*i + j + 1)/10:.1f}A {(10 - (3*i + j + 1))/10:.1f}B").replace(" ", "_") + ".png", imagens[3*i + j], [cv2.IMWRITE_PNG_COMPRESSION, 0])
    plt.show()



# 1.8 Transformação de Intensidade
Dada (a) uma imagem monocromática, transformar seu espaço de intensidades (níveis de cinza) para: 

(b) obter o negativo da imagem, ou seja, o nível de cinza 0 será convertido para 255, o nível 1 para 254 e assim por diante

(c) converter o intervalo de intensidades para [100, 200]

(d) inverter os valores dos pixels das linhas pares da imagem
Ou seja, os valores dos pixels da linha 0 serão posicionados da direita para a esquerda, os valores dos pixels da linha 2 serão posicionados da direita para a esquerda e assim por diante

(e) espelhar as linhas da metade superior da imagem na parte inferior da imagem e 

(f) aplicar um espelhamento vertical na imagem levando-se em conta todas as linhas da imagem.

In [ ]:
nome = "house"
imagem_orig = cv2.imread(input_path + nome + ".png", cv2.IMREAD_GRAYSCALE)
imagem_orig = imagem_orig.astype(np.uint16)

titulos = ["(a) Imagem original", "(b) Negativo da imagem", "(c) Imagem transformada", "(d) Linhas pares invertidas", "(e) Reflexão de linhas", "(f) Espelhamento vertical"]

#(a)
imagens = [imagem_orig]

#(b)
imagens.append(255 - imagem_orig)

#(c)
imagens.append(cv2.normalize(imagem_orig, None, alpha=100, beta=200, norm_type=cv2.NORM_MINMAX))

#(d)
temp = imagem_orig.copy()
temp[::2] = cv2.flip(temp[::2],1)
imagens.append(temp)

#(e)
temp = imagem_orig.copy()
temp[temp.shape[0]//2:] = cv2.flip(temp[:temp.shape[0]//2],0)
imagens.append(temp)

#(f)
imagens.append(cv2.flip(imagem_orig, 0))

for i in range(2):
    plt.figure(figsize=(30, 10))
    for j in range(3):
        plt.subplot(1, 3, j + 1)
        plt.imshow(imagens[3*i + j], cmap="gray", vmin=0, vmax=255)
        plt.title(f"{titulos[3*i + j]}")
        plt.axis("off")
        cv2.imwrite(output_path + "8_" + (f"{titulos[3*i + j]}").replace(" ", "_") + ".png", imagens[3*i + j], [cv2.IMWRITE_PNG_COMPRESSION, 0])
    plt.show()


# 1.9 Quantização de Imagens
Quantização refere-se ao número de níveis de cinza usados para representar uma imagem monocromática. A quantização está relacionada à profundidade de uma imagem, a qual corresponde ao número de bits necessários para armazenar a imagem. Representar uma imagem com diferentes níveis de quantização.


In [ ]:
nome = 'butterfly'
imagem_orig = cv2.imread(input_path + nome + '.png', cv2.IMREAD_GRAYSCALE)
imagem_orig = imagem_orig.astype(np.int16)

# Definir os limites para manipulação dos bits
limites = [2**i - 1 for i in range(1, 9)]

# Inicializar a lista de imagens com arrays do mesmo shape da imagem original
imagens = [np.zeros_like(imagem_orig, dtype=np.uint16) for _ in range(9)]

# Processar cada limite
for i in range(len(limites)):
    temp = np.zeros_like(imagem_orig, dtype=np.uint16)
    temp = cv2.normalize(imagem_orig, temp, alpha=0, beta=limites[i], norm_type=cv2.NORM_MINMAX)
    imagens[i] = cv2.normalize(temp, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

imagens[8] = imagem_orig

for i in range(3):
    plt.figure(figsize=(30, 10))
    for j in range(3):
        plt.subplot(1, 3, j + 1)
        plt.imshow(imagens[3*i + j], cmap='gray', vmin=0, vmax=255)
        plt.title('Imagem Original' if 3*i + j == 8 else f'{limites[3*i + j] + 1} Níveis')
        plt.axis('off')
        cv2.imwrite(output_path + "9_" + ('Imagem_Original' if 3*i + j == 8 else f'{limites[3*i + j] + 1}_Niveis') + ".png", imagens[3*i + j], [cv2.IMWRITE_PNG_COMPRESSION, 0])
    plt.show()

# 1.10 Filtragem de Imagens
A filtragem aplicada a uma imagem digital é uma operação local que altera os valores de intensi-
dade dos pixels da imagem levando-se em conta tanto o valor do pixel em questão quanto valores
de pixels vizinhos.
No processo de filtragem, utiliza-se uma operação de convolução de uma máscara pela ima-
gem. Este processo equivale a percorrer toda a imagem alterando seus valores conforme os pesos
da máscara e as intensidades da imagem.
Aplicar os filtros h1 a h11 em uma imagem digital monocromática.

In [ ]:
#definindo os filtros
h1 = [[0,0,-1,0,0],
    [0,-1,-2,-1,0],
    [-1,-2,16,-2,-1],
    [0,-1,-2,-1,0],
    [0,0,-1,0,0]]

h1 = np.array(h1)
h1 = h1.astype(np.float32)

h2 = [[1,4,6,4,1],
    [4,16,24,16,4],
    [6,24,36,24,6],
    [4,16,24,16,4],
    [1,4,6,4,1]]

h2 = np.array(h2)
h2 = h2.astype(np.float32)

h3 = [[-1,0,1],
    [-2,0,2],
    [-1,0,1]]

h3 = np.array(h3)
h3 = h3.astype(np.float32)

h4 = [[-1,-2,-1],
    [0,0,0],
    [1,2,1]]

h4 = np.array(h4)
h4 = h4.astype(np.float32)

h5 = [[-1,-1,-1],
    [-1,8,-1],
    [-1,-1,-1]]

h5 = np.array(h5)
h5 = h5.astype(np.float32)

h6 = [[1,1,1],
    [1,1,1],
    [1,1,1]]

h6 = np.array(h6)
h6 = h6.astype(np.float32)

h7 = [[-1,-1,2],
    [-1,2,-1],
    [2,-1,-1]]

h7 = np.array(h7)
h7 = h7.astype(np.float32)

h8 = [[2,-1,-1],
    [-1,2,-1],
    [-1,-1,2]]

h8 = np.array(h8)
h8 = h8.astype(np.float32)

h9 = [[1,0,0,0,0,0,0,0,0],
    [0,1,0,0,0,0,0,0,0],
    [0,0,1,0,0,0,0,0,0],
    [0,0,0,1,0,0,0,0,0],
    [0,0,0,0,1,0,0,0,0],
    [0,0,0,0,0,1,0,0,0],
    [0,0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,0,1]]

h9 = np.array(h9)
h9 = h9.astype(np.float32)

h10 = [[-1,-1,-1,-1,-1],
    [-1,2,2,2,-1],
    [-1,2,8,2,-1],
    [-1,2,2,2,-1],
    [-1,-1,-1,-1,-1]]

h10 = np.array(h10)
h10 = h10.astype(np.float32)

h11 = [[-1,-1,0],
    [-1,0,1],
    [0,1,1]]

h11 = np.array(h11)
h11 = h11.astype(np.float32)

filtros = [h1, (1/256) * h2, h3, h4, h5, (1/9) * h6, h7, h8, (1/9) * h9, (1/8) * h10, h11]

In [ ]:
nome = "peppers"
imagem_orig = cv2.imread(input_path + nome + ".png", cv2.IMREAD_GRAYSCALE)
imagem_orig = imagem_orig.astype(np.uint16)
imagens = []

for i in range(len(filtros)):
    imagem_filtrada = cv2.filter2D(imagem_orig, -1, filtros[i], borderType=cv2.BORDER_REFLECT)
    imagem_final = cv2.normalize(imagem_filtrada, imagem_final, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    imagem_final = imagem_final.astype(np.uint8)
    imagens.append(imagem_final)

temp = np.sqrt(np.square(imagens[2]) + np.square(imagens[3]))
temp = temp.astype(np.uint8)
imagens.append(cv2.normalize(temp, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX))

for i in range(4):
    plt.figure(figsize=(30, 10))
    for j in range(3):
        plt.subplot(1, 3, j + 1)
        plt.imshow(imagens[3*i + j], cmap="gray", vmin=0, vmax=255)
        plt.title("Combinação h3 e h4" if 3*i + j == 11 else f"Filtro h{(3*i + j) + 1}")
        cv2.imwrite(output_path + "10_" + ("filtro_h3_e_h4" if 3*i + j == 11 else f"filtro_h{(3*i + j) + 1}") + ".png", imagens[3*i + j], [cv2.IMWRITE_PNG_COMPRESSION, 0])
        plt.axis("off")
    plt.show()